In [1]:
setwd("../")

In [2]:
# Script D: script_D.R
library(data.table)
library(stringr)

# Load genome files
genome_files <- list.files("/expanse/lustre/projects/jhu152/naglemi/mwas/gwas",
                           pattern = "EUR", full.names = TRUE)
genome_files <- genome_files[grepl("pvar", genome_files)]

genome_files <- data.table(path = genome_files, Chr = NA)

genome_files$Chr <- str_split_fixed(genome_files$path, "chr", 2)[, 2]
genome_files$Chr <- gsub(".pvar", "", genome_files$Chr)

genome_files$Chr <- as.integer(genome_files$Chr)
genome_files <- genome_files[order(genome_files$Chr), ]

# SLURM script generation function
generate_slurm_script <- function(genome_file_index) {
  script <- paste0(
    "#!/bin/bash\n",
    "#SBATCH --partition=shared\n",
    "#SBATCH -A jhu152\n",
    "#SBATCH --nodes=1\n",
    "#SBATCH --ntasks=1\n",
    "#SBATCH --cpus-per-task=1\n",
    "#SBATCH --mem-per-cpu=32G\n",
    "#SBATCH --output=slurm_output_", genome_file_index, ".out\n",
    "#SBATCH --job-name=genome_", genome_file_index, "\n",
    "#SBATCH --time=24:00:00\n",
    #"module load conda\n",
    "conda activate mwas\n",
    "Rscript /expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/scripts/deploy_gwas2mwas_batch.R --genome_file_index=", genome_file_index, "\n"
  )
  writeLines(script, paste0("/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/slurm_scripts/genome_", genome_file_index, ".sh"))
}

In [3]:
file.exists("/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/scripts/deploy_gwas2mwas_batch.R")

[1] TRUE

In [4]:
# # Generate and submit SLURM scripts for each genome file
# for (i in 1) {
#   generate_slurm_script(i)
#   system(paste("sbatch slurm_scripts/genome_", i, ".sh"))
# }

In [5]:
# Generate and submit SLURM scripts for each genome file
#for (i in seq_len(nrow(genome_files))) {
for(i in c(2, 17)){
generate_slurm_script(i)
  system(paste("sbatch slurm_scripts/genome_", i, ".sh"))
}

In [6]:
getwd()

[1] "/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS"

In [7]:
i

[1] 22

In [8]:
file.exists("slurm_scripts/genome_1.sh")

[1] TRUE